In [34]:
import requests
import os
url = "https://huggingface.co/datasets/cbdb/cbdb-sqlite/resolve/main/latest.7z?download=true"
r = requests.get(url)
os.makedirs("./cbdb_sqlite", exist_ok=True)
with open("./cbdb_sqlite/latest.7z", "wb") as f:
    f.write(r.content)

fatal: destination path 'cbdb_sqlite' already exists and is not an empty directory.


In [35]:
# import glob
import os
latest_file = "./cbdb_sqlite/latest.7z"
latest_db = latest_file.split("/")[2].split(".")[0] + ".db"
print(latest_db)
if os.path.isfile(latest_db):
    os.remove(latest_db)

latest.db


In [36]:
# Explode latest database file
!pip install pyunpack
!pip install patool
from pyunpack import Archive
Archive(latest_file).extractall(".")

In [37]:
# Create connection to database
import sqlite3
import pandas as pd
global CONN
database_file = os.path.basename(latest_file).split(".")[0]+".db"
CONN = sqlite3.connect(database_file)

In [38]:
office_df = pd.read_sql_query("SELECT c_office_chn, c_dy FROM OFFICE_CODES", CONN).drop_duplicates().dropna()
office_alt_df = pd.read_sql_query("SELECT c_office_chn_alt, c_dy FROM OFFICE_CODES", CONN).drop_duplicates().dropna()

In [39]:
office_df

,c_office_chn,c_dy
0,未詳,15.0
1,提舉,15.0
2,三司推勘院,15.0
3,催欠司,15.0
4,三司都勾院,15.0
...,...,...
32200,訓術,20.0
32201,奉政大夫,20.0
32202,待刪除,6.0
32203,待刪除,20.0


In [40]:
# flat out c_office_chn_alt
office_alt_df['c_office_chn_alt'] = office_alt_df['c_office_chn_alt'].apply(lambda x: x.split(';'))
office_alt_df = office_alt_df.explode('c_office_chn_alt').drop_duplicates().dropna()
office_alt_df

,c_office_chn_alt,c_dy
4,三司勾院,15.0
4,都勾院,15.0
5,理欠,15.0
6,三司磨勘司,15.0
6,磨勘司,15.0
...,...,...
31732,提刑司,17.0
32186,知某州,17.0
32186,知州,17.0
32193,戶部清吏司,20.0


In [41]:
def names_massage(df, column):
  # remove content in brackets
  df[column] = df[column].str.replace("[\(（].*[\)）]", "", regex=True)
  # replace spaces with blank
  df[column] = df[column].str.replace("\\s", "", regex=True)
  # remove single-character name
  df = df[df[column].apply(lambda x: len(x)>1)]
  # filter out weird characters
  df = df[df[column].str.contains('\*|、|某|刪除|未詳|\[|\]|【|】')==False]
  df = df[df[column].str.contains('.*[a-zA-Z]+.*')==False]
  df.drop_duplicates().dropna()
  return df

In [42]:
filtered_office_df = names_massage(office_df, 'c_office_chn')
filtered_office_alt_df = names_massage(office_alt_df, 'c_office_chn_alt')

In [43]:
filtered_office_df.columns = ['office_name', 'dynasty_index']
filtered_office_alt_df.columns = ['office_name', 'dynasty_index']

In [44]:
filtered_office_df

,office_name,dynasty_index
1,提舉,15.0
2,三司推勘院,15.0
3,催欠司,15.0
4,三司都勾院,15.0
5,三司都理欠司,15.0
...,...,...
32198,通奉大夫,20.0
32199,閘官,20.0
32200,訓術,20.0
32201,奉政大夫,20.0


In [45]:
filtered_office_alt_df

,office_name,dynasty_index
4,三司勾院,15.0
4,都勾院,15.0
5,理欠,15.0
6,三司磨勘司,15.0
6,磨勘司,15.0
...,...,...
31686,治中,17.0
31732,提刑司,17.0
32186,知州,17.0
32193,戶部清吏司,20.0


In [46]:
# combine df
final_df = pd.concat([filtered_office_df, filtered_office_alt_df], keys=('office_name', 'dynasty_index')).reset_index(drop=True).drop_duplicates().dropna()

In [47]:
# Sort by descending order of the length
final_df = final_df.sort_values(by="office_name", key=lambda x: -x.str.len())
final_df

,office_name,dynasty_index
2558,同管勾成都府利州陝西等路茶事兼提舉陝西等路買馬監牧公事,15.0
2557,同提舉成都府利州陝西等路茶場公事兼提舉陝西買馬監牧公事,15.0
2556,都大提舉成都府利州等路茶事兼提舉四川等路買馬監牧公事,15.0
9421,管領本投下大都等路打捕鷹房諸色人匠都總管府提控案牘,18.0
9417,管領本投下大都等路打捕鷹房諸色人匠都總管府達魯花赤,18.0
...,...,...
33484,馬帥,15.0
17616,曹道,20.0
33482,內職,15.0
8633,部役,18.0


In [48]:
final_df.to_csv('cbdb_entity_offices.csv', index=False)

In [49]:
# close the db
CONN.close()